In [1]:
# Check GPU
!nvidia-smi

Fri May 31 10:43:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:00:05.0 Off |                    0 |
| N/A   31C    P0              41W / 300W |      0MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
# Import packages
from IPython.display import display, Image
from IPython import display
import csv
import os
import json

In [3]:
# Import roboflow
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="YgqaFYQYMXIDYOCYoY2O") # Fill in your person api_key
project = rf.workspace("footage").project("seafloor_footage") # Fill in your workspace and project name
model = project.version("3").model # Fill in the number of the version

display.clear_output()
print("Successfully extracted data from roboflow")

Successfully extracted data from roboflow


In [7]:
def create_output_dir(base_dir): # Create new dir
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
        return base_dir
    else:
        counter = 2
        while True:
            new_dir = f"{base_dir}{counter}"
            if not os.path.exists(new_dir):
                os.makedirs(new_dir)
                return new_dir
            counter += 1

base_output_dir = "/cache/album/cache/kso-benjamin/bucket/kso/notebooks/test/predictions/objd-pred" # Define base dir
output_dir = create_output_dir(base_output_dir)

input_directory = "/cache/album/cache/kso-benjamin/bucket/kso/notebooks/test/Seafloor_footage-1/Raw/" # Define dir with raw images

for filename in os.listdir(input_directory): # Iterate over the files
    filepath = os.path.join(input_directory, filename) # Get the full path 
    
    if filepath.lower().endswith(('.png', '.jpg', '.jpeg')): # Check format

        prediction = model.predict(filepath, confidence=40, overlap=30) # Perform prediction on the image
        
        prediction_data = prediction.json() # Get the JSON results
        
        csv_filename = os.path.splitext(filename)[0] + "_prediction.csv" # Write to csv
        csv_filepath = os.path.join(output_dir, csv_filename)
        
        with open(csv_filepath, mode='w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            
            header = ["x", "y", "width", "height", "confidence", "class", "class_id", "detection_id", "image_path", "prediction_type"] # Write the header row
            csv_writer.writerow(header)
            
            for idx, pred in enumerate(prediction_data['predictions']):  # Write each prediction as a row
                row = [
                    pred.get('x', ''),
                    pred.get('y', ''),
                    pred.get('width', ''),
                    pred.get('height', ''),
                    pred.get('confidence', ''),
                    pred.get('class', ''),
                    pred.get('class_id', ''),
                    pred.get('detection_id', ''),
                    pred.get('image_path', ''),
                    pred.get('prediction_type', '')
                ]
                csv_writer.writerow(row)
        
        labeled_prediction = model.predict(filepath, confidence=40, overlap=30, labels=True) # Perform prediction with labels and save the result
        
        output_image_path = os.path.join(output_dir, os.path.splitext(filename)[0] + "_prediction.jpg") # Construct the output file path for labeled prediction image
        
        labeled_prediction.save(output_image_path) # Save the labeled prediction result to an image file
        
        display.clear_output()
